In [2]:
# %pip install pydantic openai tqdm pandas -U

In [3]:
# %pip install jinja2

In [1]:
import json
from typing import Literal
from openai import OpenAI
from pydantic import BaseModel, Field
import pandas as pd

from tqdm import tqdm

In [ ]:
api_key = '<api_key>'

In [60]:
client = OpenAI(api_key=api_key)

In [61]:
model_str="gpt-4o-2024-11-20"

In [62]:
system_prompt = """
Ты выступаешь в роли судьи для оценки качества генерации текста заключения офтальмолога, созданного языковой моделью на основании параметров глазного дна. Проверь ответ по четырём критериям, выстави оценки и обоснуй их.

**Входные данные:**
- JSON с исходными параметрами глазного дна
- Текстовое описание, сгенерированное языковой моделью

---

**Критерий №1: Совпадение входных параметров с текстовым описанием**  
Проверь, чтобы каждый параметр из JSON был верно отражён в тексте. Допускается перефразирование (например, «нормальный» → «нормального»).

**Оценка:**  
- 1 балл — все параметры отражены корректно, без расхождений  
- 0 баллов — есть хотя бы одно расхождение

Приведи пример расхождения при оценке 0.

---

**Критерий №2: Отсутствие вводных конструкций и лишних фраз**  
Текст должен содержать только медицинское описание. Не допускаются вводные конструкции или пояснительные фразы (например: «в данном случае», «следует отметить», «как известно» и т.п.), а также фразы от языковой модели (например: «конечно», «текст описания» и т.п.).

**Оценка:**  
- 1 балл — нет вводных конструкций  
- 0 баллов — есть хотя бы одна

Приведи пример лишней фразы при оценке 0.

---

**Критерий №3: Соблюдение порядка описания структур**  
Порядок должен быть следующим:

1. **Диск зрительного нерва (ДЗН)**  
   - Цвет  
   - Монотонность окраски  
   - Размер  
   - Форма  
   - Границы  

2. **Экскавация ДЗН**  
   - Размер экскавации  
   - Положение экскавации  
   - Сосудистый пучок  
   - Патологические изменения (при наличии)

3. **Сосуды глазного дна**  
   - Ход  
   - Извитость  
   - Бифуркация  
   - Калибр  
   - А/В индекс  
   - Патологии (если есть)

4. **Макула**  
   - Макулярный рефлекс  
   - Фовеолярный рефлекс

5. **Периферия глазного дна**  
   - Только патологические изменения (если есть)

**Оценка:**  
- 1 балл — порядок строго соблюдён  
- 0 баллов — есть отклонения

Укажи, где нарушен порядок при оценке 0.

---

**Критерий №4: Плавность и естественность языка**  
Оцени текст с точки зрения профессионального языка врача-офтальмолога.

**Оценка:**  
От 0 (очень неестественно) до 5 (максимально естественно и профессионально)

Кратко обоснуй оценку, указав стилистически слабые места, если есть.
"""


In [63]:
class EvaluationResult(BaseModel):
    param_match_score: Literal[0, 1] = Field(..., description="1, если все параметры из JSON корректно отражены в тексте; 0, если есть хотя бы одно расхождение")
    no_intro_phrases_score: Literal[0, 1] = Field(..., description="1, если в тексте нет вводных конструкций и лишних фраз; 0 — если есть хотя бы одна")
    structure_order_score: Literal[0, 1] = Field(..., description="1, если порядок описания структур соблюдён строго; 0 — если есть отклонения")
    language_naturalness_score: Literal[0, 1, 2, 3, 4, 5] = Field(..., description="Оценка от 0 до 5 за плавность и естественность медицинского языка")

    param_match_comment: str = Field(..., description="Комментарий к оценке по параметрам; указать пример несоответствия при необходимости")
    no_intro_phrases_comment: str = Field(..., description="Комментарий к оценке по вводным конструкциям; привести пример при необходимости")
    structure_order_comment: str = Field(..., description="Комментарий к оценке порядка структур; указать, где нарушен порядок при необходимости")
    language_naturalness_comment: str = Field(..., description="Комментарий к оценке плавности и естественности языка; обосновать балл")

In [2]:
with open("TEST_prompts.json", "r", encoding="utf-8") as f:
    loaded_samples = json.load(f)

len(loaded_samples)

100

## **Scoring**

In [65]:
def scoring_function(data_path, type):
    df = pd.read_csv(data_path)
    texts = df['text']

    data = [(loaded_samples[i], texts[i]) for i in range(len(loaded_samples))]

    dataframe_elements = []

    for item in data:
        json_data = item[0]
        llm_text = item[1]

        user_prompt = f"""Исходный JSON:
    {json_data}

    Сгенерированный текст:
    {llm_text}
    """

        completion = client.beta.chat.completions.parse(
            model=model_str,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt},
            ],
            response_format=EvaluationResult,
        )

        json_answer = completion.choices[0].message.content
        llm_dict = json.loads(json_answer)
        dataframe_elements.append(llm_dict)

    df = pd.DataFrame(dataframe_elements)
    df.to_csv(f"scores/{type}_scores.csv")

In [66]:
scoring_metas = [
    ('data/BioMistal_data.csv', 'BioMistral'),
    ('data/RuAdapt_data.csv', 'RuAdapt'),
    ('data/Saiga_data.csv', 'Saiga'),
    ('data/Yandex_data.csv', 'Yandex'),
    ('data/TableLlama_data.csv', 'TableLlama'),
]

for meta in scoring_metas:
    print(meta)
    scoring_function(meta[0], meta[1])

('data/BioMistal_data.csv', 'BioMistral')
('data/RuAdapt_data.csv', 'RuAdapt')
('data/Saiga_data.csv', 'Saiga')
('data/Yandex_data.csv', 'Yandex')
('data/TableLlama_data.csv', 'TableLlama')
